In [253]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import warnings
import json

In [254]:
warnings.filterwarnings('ignore')

In [255]:
with open('test_one_session.json', 'r') as file:
     data = [json.loads(line) for line in file]

In [256]:
df1 = []

for user_id, info in data[0].items():
    df1.append({
        'user_id': user_id,
        'vacancy_id_preds': info
    })
    
df1 = pd.DataFrame(df1)
df1.head()

,user_id,vacancy_id_preds
0,u_263456,"[v_2466770, v_2223951, v_624968, v_69160, v_13..."
1,u_510393,"[v_869632, v_1662776, v_1189776, v_2519382, v_..."
2,u_323157,"[v_2363709, v_850398, v_416301, v_949145, v_19..."
3,u_301554,"[v_2431072, v_2014471, v_256240, v_890055, v_4..."
4,u_186244,"[v_1887316, v_130779, v_942083, v_2075948, v_1..."


In [257]:
full_validation = pd.read_csv('true_one_session.csv')
full_validation.head()

,user_id,index_test,index_true
0,u_263456,v_1900367,"['v_1711113', 'v_2501360', 'v_141541', 'v_4846..."
1,u_510393,v_242868,"['v_1188912', 'v_2381375', 'v_2092069', 'v_269..."
2,u_1072870,v_2689042,"['v_661027', 'v_1205504', 'v_970840', 'v_11281..."
3,u_323157,v_447253,"['v_2259541', 'v_705963', 'v_360217', 'v_46998..."
4,u_301554,v_14401,"['v_1488606', 'v_1134517', 'v_2047069', 'v_240..."


In [258]:
merged_full_validation = pd.merge(full_validation,
                                  df3,
                                  on='user_id',
                                  how='inner')

merged_full_validation.head()

,user_id,index_test,index_true,vacancy_id_preds
0,u_263456,v_1900367,"['v_1711113', 'v_2501360', 'v_141541', 'v_4846...","[v_1462, v_2165659, v_2169068, v_719500, v_103..."
1,u_510393,v_242868,"['v_1188912', 'v_2381375', 'v_2092069', 'v_269...","[v_1791508, v_1529820, v_2227902, v_2230791, v..."
2,u_323157,v_447253,"['v_2259541', 'v_705963', 'v_360217', 'v_46998...","[v_1775588, v_47653, v_2122716, v_381383, v_26..."
3,u_301554,v_14401,"['v_1488606', 'v_1134517', 'v_2047069', 'v_240...","[v_45854, v_708560, v_166650, v_1646503, v_591..."
4,u_186244,v_1092714,"['v_985761', 'v_2199976', 'v_2384625', 'v_1160...","[v_2510503, v_2291253, v_1851808, v_2443823, v..."


In [259]:
merged_full_validation['index_true'] = merged_full_validation['index_true']\
                                .apply(lambda x: ast.literal_eval(x))\
                                .apply(lambda x: [id.strip(" '[]") for id in x])

In [260]:
merged_full_validation.head()

,user_id,index_test,index_true,vacancy_id_preds
0,u_263456,v_1900367,"[v_1711113, v_2501360, v_141541, v_484626, v_2...","[v_1462, v_2165659, v_2169068, v_719500, v_103..."
1,u_510393,v_242868,"[v_1188912, v_2381375, v_2092069, v_269200, v_...","[v_1791508, v_1529820, v_2227902, v_2230791, v..."
2,u_323157,v_447253,"[v_2259541, v_705963, v_360217, v_469988, v_25...","[v_1775588, v_47653, v_2122716, v_381383, v_26..."
3,u_301554,v_14401,"[v_1488606, v_1134517, v_2047069, v_2406644, v...","[v_45854, v_708560, v_166650, v_1646503, v_591..."
4,u_186244,v_1092714,"[v_985761, v_2199976, v_2384625, v_1160174, v_...","[v_2510503, v_2291253, v_1851808, v_2443823, v..."


In [263]:
merged_full_validation.to_csv('full_validation_one_session.csv', index=False)

In [264]:
vacancy = pd.read_csv('vacancies_full_keyskills.csv')
vacancy.dropna(subset=['keySkills_str'], inplace=True)
vacancy.head()

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder,keySkills_str
0,v_862116,Смотритель музейный,c_162972,"['Уверенный пользователь пк', 'Работа в команд...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный,32,4181,69972,5,5,1,Уверенный пользователь пк Работа в команде Уме...
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"['Активные продажи', 'Холодные продажи', 'Кред...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицами,17,6731,120750,5,5,1,Активные продажи Холодные продажи Кредитные до...
2,v_312507,Торговый представитель (г. Абакан),c_206699,"['Продуктивность', 'Клиентоориентированность',...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Торговый представитель,39,6487,118557,5,5,2,Продуктивность Клиентоориентированность Работа...
3,v_2323706,Менеджер по качеству,c_244141,"['Контроль качества', 'Производственный контро...",88000.0,NaN,RUR,a_5920,ar_64,full,fullDay,between1And3,Менеджер по качеству,66,5469,160161,5,5,2,Контроль качества Производственный контроль На...
4,v_704266,Менеджер по продажам / Sales manager,c_192038,"['Активные продажи', 'Поиск и привлечение клие...",30000.0,100000.0,RUR,a_5398,ar_71,full,fullDay,between1And3,Менеджер по продажам,74,4888,102268,5,5,2,Активные продажи Поиск и привлечение клиентов ...


In [327]:
def predict_with_params(user_df,
                        vac_df,
                        col_for_predict,
                        col_from_predict,
                        col_transform,
                        features,
                        n_neighbors=30):
    
    user_neighbors = {}
    
    for user_id in tqdm(user_df['user_id'].unique(), desc='Processing users'):
        
        features_copy = features.copy()
        user_data = user_df[user_df['user_id'] == user_id]
        vacancy_id = user_data[col_for_predict].values[0]
        vac_need_ids = list(user_df[col_from_predict].values[0]) 
        need_vac_df = vac_df[vac_df['vacancy_id'].isin(vac_need_ids)].copy() 
        if vacancy_id not in vac_need_ids:
            vacancy_to_add = vac_df[vac_df['vacancy_id'] == vacancy_id]
            need_vac_df = pd.concat([need_vac_df, vacancy_to_add])
        
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(need_vac_df[col_transform])
        knn_model_str = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
        knn_model_str.fit(X)
        
        combined_df = pd.DataFrame()
        
        try:
            query = vectorizer.transform(need_vac_df[need_vac_df['vacancy_id'] == vacancy_id][col_transform])
        except Exception:
            continue
            
        distances, indices = knn_model_str.kneighbors(query, n_neighbors=n_neighbors)
        vacancy_neighbors = need_vac_df.iloc[indices[0]].copy()
        vacancy_neighbors['distance_str'] = distances[0]
        combined_df = pd.concat([combined_df, vacancy_neighbors])
        
        target_vacancy_df = need_vac_df[need_vac_df['vacancy_id'] == vacancy_id].copy()
        
        if 'compensation_from' in features_copy:
            features_init = features.copy()
            features_copy.remove('compensation_from')
            combined_df.dropna(subset=['compensation_from'], inplace=True)

        onehot_df = pd.get_dummies(combined_df[features_copy], prefix=features_copy)
        onehot_target_vac_df = pd.get_dummies(target_vacancy_df[features_copy], prefix=features_copy)
        
        missing_cols = set(onehot_df.columns) - set(onehot_target_vac_df.columns)
        for col in missing_cols:
            onehot_target_vac_df[col] = 0
        
        cat_columns = onehot_df.columns

        combined_df = pd.concat([combined_df, onehot_df], axis=1)
        combined_df.drop(features_copy, axis=1, inplace=True)
        combined_df = combined_df.reset_index(drop=True)
        
        target_vacancy_df = pd.concat([target_vacancy_df, onehot_target_vac_df], axis=1)
        target_vacancy_df.drop(features_copy, axis=1, inplace=True)

        if 'compensation_from' in features_init:
            cat_columns = cat_columns.tolist() + ['compensation_from']
            combined_df['compensation_from'] = (combined_df['compensation_from'] - combined_df['compensation_from'].min()) \
                        / (combined_df['compensation_from'].max() - combined_df['compensation_from'].min())
            target_vacancy_df['compensation_from'] = (target_vacancy_df['compensation_from'] - combined_df['compensation_from'].min()) \
                        / (combined_df['compensation_from'].max() - combined_df['compensation_from'].min())
            knn_model_cat = NearestNeighbors(n_neighbors=len(combined_df), metric='euclidean')
        else:
            knn_model_cat = NearestNeighbors(n_neighbors=len(combined_df), metric='hamming')
            
        try:
            knn_model_cat.fit(combined_df[cat_columns])
        except Exception as e:
            print(e)
            print(user_id)
            continue

        query = combined_df[cat_columns].values
        distances, indices = knn_model_cat.kneighbors(query, n_neighbors=len(combined_df))
        combined_df['distance_cat'] = distances.mean(axis=1)
        combined_df['distance_cat'] = (combined_df['distance_cat'] - combined_df['distance_cat'].min()) \
                        / (combined_df['distance_cat'].max() - combined_df['distance_cat'].min())
        combined_df['distance_str'] = (combined_df['distance_str'] - combined_df['distance_str'].min()) \
                        / (combined_df['distance_str'].max() - combined_df['distance_str'].min())
        combined_df['average_distance'] = combined_df[['distance_str', 'distance_cat']].mean(axis=1)
        combined_df.sort_values(by='average_distance', inplace=True)
        
        user_neighbors[user_id] = {
            'vacancy_id': vacancy_id,
            'predict': combined_df.head(5)['vacancy_id'].tolist()
        }
        
    return user_neighbors

In [328]:
initial_predict = predict_with_params(
    merged_full_validation,
    vacancy,
    'index_test',
    'vacancy_id_preds',
    'keySkills_str',
    ['workExperience', 'employment', 'compensation_from']
)

Processing users: 100%|███████████████████████| 962/962 [04:43<00:00,  3.39it/s]


In [329]:
predict = []

for user_id, info in initial_predict.items():
    predict.append({
        'user_id': user_id,
        'vacancy_id': info['vacancy_id'],
        'vacancy_id_preds': info['predict']
    })

predict_df = pd.DataFrame(predict)
predict_df.head()

,user_id,vacancy_id,vacancy_id_preds
0,u_263456,v_1900367,"[v_1900367, v_1915162, v_1514089, v_2683051, v..."
1,u_510393,v_242868,"[v_2289714, v_2403272, v_242868, v_1581953, v_..."
2,u_323157,v_447253,"[v_1795950, v_325366, v_1596742, v_2420819, v_..."
3,u_301554,v_14401,"[v_14401, v_1383335, v_1442985, v_874780, v_88..."
4,u_186244,v_1092714,"[v_1092714, v_325748, v_1393064, v_494996, v_3..."


In [330]:
merged_full_validation.head()

,user_id,index_test,index_true,vacancy_id_preds
0,u_263456,v_1900367,"[v_1711113, v_2501360, v_141541, v_484626, v_2...","[v_1462, v_2165659, v_2169068, v_719500, v_103..."
1,u_510393,v_242868,"[v_1188912, v_2381375, v_2092069, v_269200, v_...","[v_1791508, v_1529820, v_2227902, v_2230791, v..."
2,u_323157,v_447253,"[v_2259541, v_705963, v_360217, v_469988, v_25...","[v_1775588, v_47653, v_2122716, v_381383, v_26..."
3,u_301554,v_14401,"[v_1488606, v_1134517, v_2047069, v_2406644, v...","[v_45854, v_708560, v_166650, v_1646503, v_591..."
4,u_186244,v_1092714,"[v_985761, v_2199976, v_2384625, v_1160174, v_...","[v_2510503, v_2291253, v_1851808, v_2443823, v..."


In [331]:
validation = merged_full_validation[['user_id', 'index_test', 'index_true']].copy()
validation.head()

,user_id,index_test,index_true
0,u_263456,v_1900367,"[v_1711113, v_2501360, v_141541, v_484626, v_2..."
1,u_510393,v_242868,"[v_1188912, v_2381375, v_2092069, v_269200, v_..."
2,u_323157,v_447253,"[v_2259541, v_705963, v_360217, v_469988, v_25..."
3,u_301554,v_14401,"[v_1488606, v_1134517, v_2047069, v_2406644, v..."
4,u_186244,v_1092714,"[v_985761, v_2199976, v_2384625, v_1160174, v_..."


In [332]:
res = pd.merge(predict_df, validation, on='user_id', how='inner')
res.rename(columns={'index_true': 'vacancy_id_true'}, inplace=True)
res.head()

,user_id,vacancy_id,vacancy_id_preds,index_test,vacancy_id_true
0,u_263456,v_1900367,"[v_1900367, v_1915162, v_1514089, v_2683051, v...",v_1900367,"[v_1711113, v_2501360, v_141541, v_484626, v_2..."
1,u_263456,v_1900367,"[v_1900367, v_1915162, v_1514089, v_2683051, v...",v_2678071,"[v_1903334, v_2566704, v_1775258, v_82956, v_2..."
2,u_263456,v_1900367,"[v_1900367, v_1915162, v_1514089, v_2683051, v...",v_459840,"[v_580765, v_2397911, v_1952634, v_2255597, v_..."
3,u_263456,v_1900367,"[v_1900367, v_1915162, v_1514089, v_2683051, v...",v_1078795,"[v_1900367, v_1844848, v_703243, v_2094299, v_..."
4,u_263456,v_1900367,"[v_1900367, v_1915162, v_1514089, v_2683051, v...",v_2142328,"[v_2443974, v_464599, v_767341, v_190971, v_21..."


In [333]:
%run '../metric/metric_rank'

In [334]:
def get_similarity(row, data, threshold):
    return calculate_similarity(row, data, threshold)

In [335]:
small_res = res.sample(n=100, random_state=42)
small_res.reset_index(drop=True, inplace=True)

In [370]:
tqdm.pandas()

small_res['result'] = small_res.progress_apply(
    get_similarity,
    data=vacancy,
    threshold=0.72,
    axis=1
)

100%|█████████████████████████████████████████| 100/100 [29:16<00:00, 17.56s/it]


In [372]:
small_res.head()

,user_id,vacancy_id,vacancy_id_preds,index_test,vacancy_id_true,result
0,u_982973,v_2613295,"[v_2613295, v_310517, v_883974, v_5910, v_25748]",v_2613295,"[v_694655, v_16379, v_785329, v_1316686, v_393...",0.977870
1,u_516726,v_497984,"[v_497984, v_1349598, v_422330, v_2301973, v_2...",v_497984,"[v_886725, v_1971479, v_2387220, v_2277185, v_...",0.815608
2,u_465908,v_2097564,"[v_2434087, v_565866, v_849171, v_1741509, v_3...",v_2097564,"[v_1618983, v_25054, v_2617472, v_2629427, v_5...",0.037800
3,u_471977,v_1317387,"[v_820418, v_581703, v_2645924, v_2634526, v_2...",v_1317387,"[v_1998257, v_1726273, v_471168, v_2397289, v_...",0.114750
5,u_1025504,v_2306585,"[v_1560504, v_310199, v_1405256, v_2487230, v_...",v_2306585,"[v_2590550, v_2655128, v_1888267, v_2708457, v...",0.519023


In [373]:
small_res['result'].median()

0.74300625

In [376]:
res.iloc[0]['vacancy_id_preds']  

['v_1900367', 'v_1915162', 'v_1514089', 'v_2683051', 'v_141728']

In [377]:
vacancy[vacancy['vacancy_id'] == 'v_1900367']

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder,keySkills_str
1196710,v_1900367,Менеджер по продажам B2B-услуг (удаленно),c_189657,"['B2b маркетинг', 'Телефонные переговоры', 'Cr...",130000.0,200000.0,RUR,a_1756,ar_41,full,remote,between1And3,Менеджер по продажам B2B-услуг,41,842,99621,5,3,2,B2b маркетинг Телефонные переговоры Crm Активн...


In [378]:
vacancy[vacancy['vacancy_id'] == 'v_1915162']

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder,keySkills_str
887568,v_1915162,Менеджер по оптовым продажам/менеджер по продажам,c_63663,"['B2b маркетинг', 'Навыки продаж', 'Ведение пе...",50000.0,320000.0,RUR,a_5618,ar_72,full,fullDay,between1And3,Менеджер по оптовым продажам,75,5133,238540,5,5,2,B2b маркетинг Навыки продаж Ведение переговоро...


In [379]:
vacancy[vacancy['vacancy_id'] == 'v_1514089']

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder,keySkills_str
1123969,v_1514089,Менеджер по продажам логистических услуг,c_247916,"['Поиск и привлечение клиентов', 'Холодные про...",50000.0,300000.0,RUR,a_4284,ar_23,full,fullDay,between1And3,Менеджер по продажам логистических услуг,21,3651,164354,5,5,2,Поиск и привлечение клиентов Холодные продажи ...


In [380]:
# версия для дальнейшего совмещения
from sklearn.preprocessing import MinMaxScaler

def recomend(name: str, compensation_from: int, area_id: str, workExperience: str, keySkills: list):
    data_vacancy = vacancy
    
#     self.data_vacancy = data_vacancy
#     data_vacancy = data_vacancy.loc[data_vacancy['area_id'] == area_id]

#     if len(data_vacancy)>self.nlp_cluster_size:
#         data_vacancy = data_vacancy.loc[data_vacancy['workExperience'] <= experience_code(workExperience)]
#     if len(data_vacancy)>self.nlp_cluster_size: 
#         data_vacancy = data_vacancy.loc[data_vacancy['compensation_from'] >= compensation_from * 0.8]

#     names = list(data_vacancy['name'])
#     res = []
#     if len(names)<5:
#         names = list(self.data_vacancy['name'])
#     if len(names)>self.nlp_cluster_size:
#         random.shuffle(names)
#         names = names[:self.nlp_cluster_size]

#     elif len(names)<50 and len(names)>=5:
#         for i in names:
#             res.extend(self.names_d[i])
#         return res

#     a = find_nearest_emb(name, names, self.model)
#     for i in a:
#         res.extend(self.names_d[i])
        
    res = data[0]['u_263456']
        
    if len(keySkills) == 0:
        return res[:10]
    
    keySkills_str = [skill.lower() for skill in keySkills]
    keySkills_str = ' '.join(keySkills_str)
    
    row = pd.DataFrame({
        'clean_name': [name],
        'compensation_from': [compensation_from],
        'area_regionId': [area_id],
        'workExperience': [workExperience],
        'keySkills': [keySkills],
        'keySkills_str': [keySkills_str],
    })
              
    need_vac_df = data_vacancy[data_vacancy['vacancy_id'].isin(res)].copy()
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(need_vac_df['keySkills_str'])
    
    if len(need_vac_df) > 30:
        n_neighbors = 30
    else:
        n_neighbors = len(need_vac_df)

    knn_model_str = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    knn_model_str.fit(X)

    combined_df = pd.DataFrame()

    try:
        query = vectorizer.transform(row['keySkills_str'])
    except Exception:
        return res[:10]
        
    distances, indices = knn_model_str.kneighbors(query, n_neighbors=n_neighbors)
    vacancy_neighbors = need_vac_df.iloc[indices[0]].copy()
    vacancy_neighbors['distance_str'] = distances[0]
    combined_df = pd.concat([combined_df, vacancy_neighbors])
    
    features = ['workExperience']
    combined_df.dropna(subset=['compensation_from'], inplace=True)
    onehot_df = pd.get_dummies(combined_df[features], prefix=features)
    onehot_target_vac_df = pd.get_dummies(row[features], prefix=features)

    missing_cols = set(onehot_df.columns) - set(onehot_target_vac_df.columns)
    for col in missing_cols:
        onehot_target_vac_df[col] = 0

    cat_columns = onehot_df.columns
    combined_df = pd.concat([combined_df, onehot_df], axis=1)
    combined_df.drop(features, axis=1, inplace=True)
    combined_df = combined_df.reset_index(drop=True)

    row = pd.concat([row, onehot_target_vac_df], axis=1)
    row.drop(features, axis=1, inplace=True)
    
    scaler = MinMaxScaler()
    combined_df[['compensation_from']] = scaler.fit_transform(combined_df[['compensation_from']])
    row[['compensation_from']] = scaler.transform(row[['compensation_from']])

    knn_model_cat = NearestNeighbors(n_neighbors=len(combined_df), metric='euclidean')

    try:
        knn_model_cat.fit(combined_df[cat_columns])
    except Exception as e:
        print(e)
        return res[:10]

    query = combined_df[cat_columns].values
    distances, indices = knn_model_cat.kneighbors(query, n_neighbors=len(combined_df))
    combined_df['distance_cat'] = distances.mean(axis=1)

    scaler_cat = MinMaxScaler()
    combined_df[['distance_cat']] = scaler_cat.fit_transform(combined_df[['distance_cat']])
    scaler_str = MinMaxScaler()
    combined_df[['distance_str']] = scaler_str.fit_transform(combined_df[['distance_str']])
    combined_df['average_distance'] = combined_df[['distance_str', 'distance_cat']].mean(axis=1)
    combined_df.sort_values(by='average_distance', inplace=True)

    user_neighbors = []
    user_neighbors = combined_df.head(5)['vacancy_id'].tolist()

    return user_neighbors